In [1]:
!pip install stanza


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
import stanza
stanza.download('ru')
nlp = stanza.Pipeline("ru", package="Taiga")

2023-10-21 19:53:47 INFO: Downloading default packages for language: ru (Russian) ...
2023-10-21 19:53:48 INFO: File exists: C:\Users\GL_24_12_2019\stanza_resources\ru\default.zip
2023-10-21 19:53:52 INFO: Finished downloading models and saved to C:\Users\GL_24_12_2019\stanza_resources.
2023-10-21 19:53:52 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-10-21 19:53:52 INFO: Loading these models for language: ru (Russian):
| Processor | Package        |
------------------------------
| tokenize  | taiga          |
| pos       | taiga_charlm   |
| lemma     | taiga_nocharlm |
| depparse  | taiga_charlm   |

2023-10-21 19:53:52 INFO: Using device: cpu
2023-10-21 19:53:52 INFO: Loading: tokenize
2023-10-21 19:53:52 INFO: Loading: pos
2023-10-21 19:53:53 INFO: Loading: lemma
2023-10-21 19:53:53 INFO: Loading: depparse
2023-10-21 19:53:53 INFO

In [38]:
from stanza.utils.conll import CoNLL
import json
import os
import re

In [ ]:
os.mkdir("conllu")

In [34]:
os.mkdir("conllu/connlu_with_meta")

In [85]:
# конвертируем .txt в .conllu, попутно пропарсив текст Stanz'ой
def conllu_vert(filename):
    with open(filename, 'r', encoding = 'UTF-8') as f: 
        fread = f.readlines()
        metadata = fread[0]
        full_text = ''
        for n in range(2, len(fread)-1):
            full_text = full_text + fread[n]
    full_text = re.sub(r"[\u00AD]", "", full_text)
    doc = nlp(full_text)
    conllu_fname = os.path.join('conllu', filename.split('.')[0]+'.conllu')
    metadata = json.loads(metadata)
    with open(conllu_fname, "w", encoding = "UTF-8") as f:
        f.write(f"# metadata = Название: {metadata['fic_name']}, автор: {metadata['author']}, ссылка: {metadata['link']}\n")
    CoNLL.write_doc2conll(doc, conllu_fname, mode="a")


In [102]:
os.chdir("C:\\Users\\GL_24_12_2019\\YandexDisk\\NLP_hws\\\\better_corpus")

In [103]:
import os
files = []
for file in os.listdir():
    if file.endswith('.txt'):
        files.append(file)

In [104]:
for file in files: 
    conllu_vert(file)

In [105]:
import nltk
from string import punctuation

In [106]:
with open("merged.txt", "wb") as result_file:
    for f in files:
        with open(f, "rb") as file:
            result_file.write(file.read())

In [107]:
with open('merged.txt', 'r', encoding='UTF-8') as file:
    text = file.read()
    smth = nltk.word_tokenize(text)

In [108]:
tokens_quantity = [token for token in smth if token not in punctuation]
print(f"Всего токенов в нашем корпусе приблизительно {len(tokens_quantity)}")

Всего токенов в нашем корпусе приблизительно 228129


In [109]:
# занимаемся добавлением метадаты формата название-автор-ссылка в conllu файлы
def add_metadata(file):
    with open(file, 'r', encoding="UTF-8") as f:
        txt = f.readlines()
        meta = txt[0]
        f.seek(0)
        txt = f.readlines()[1:]
        meta_fname = os.path.join('conllu_with_meta', file.split('.')[0]+'.conllu')
        with open(meta_fname, 'w', encoding="UTF-8") as fw:
            for lines in txt:
                if lines[0:9] != '# sent_id':
                    fw.write(lines)
                elif lines[0:9] == '# sent_id':
                    fw.write(lines)
                    fw.write(meta)

In [110]:
os.chdir("conllu")

In [111]:
files = []
for file in os.listdir():
    if file.endswith('.conllu'):
        files.append(file)

In [112]:
for file in files:
    add_metadata(file)